In [5]:
import json
import requests as req

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, DateTime
from sqlalchemy.orm import Session

In [6]:
# url = 'https://api.usaspending.gov/api/v1/awards/'
url = 'https://api.usaspending.gov/api/v1/awards/?limit=500&page=1'

params = {
    "filters": [
        {
            "field": "date_signed",
            "operation": "greater_than_or_equal",
            "value": "2017-01-01"
        },
        {
            "field": "date_signed",
            "operation": "less_than",
            "value": "2017-12-31"
        },
        {
            "field": "category",
            "operation": "equals",
            "value": "contract"
        },
        {
            "field": "awarding_agency__toptier_agency__cgac_code",
            "operation": "equals",
            "value": "012"
        }
    ]
}

# us_data = req.post(url , json =params).json()
us_data = req.get(url, json = params).json()

In [7]:
print(json.dumps(us_data, indent=4, sort_keys=True))

{
    "page_metadata": {
        "count": 45832,
        "current": "https://api.usaspending.gov/api/v1/awards/?limit=500&page=1",
        "has_next_page": true,
        "has_previous_page": false,
        "next": "https://api.usaspending.gov/api/v1/awards/?limit=500&page=2",
        "page": 1,
        "previous": null
    },
    "results": [
        {
            "awarding_agency": {
                "id": 158,
                "office_agency": null,
                "subtier_agency": {
                    "abbreviation": "RHS",
                    "name": "Rural Housing Service",
                    "subtier_code": "12E3"
                },
                "toptier_agency": {
                    "abbreviation": "USDA",
                    "cgac_code": "012",
                    "fpds_code": "1200",
                    "name": "Department of Agriculture"
                },
                "toptier_flag": false
            },
            "category": "contract",
            "certified_date

In [8]:
# create awards class
class Awards(Base):
    __tablename__ = 'awards'
    id = Column(Integer, primary_key=True)
    awarding_agency = Column(String(255))
    date_signed = Column(String(255))
    recipient_name = Column(String(255))
    recipient_zip = Column(String(255))
    total_obligation = Column(Float)

In [9]:
# Establish Connection to database
engine = create_engine(f'sqlite:///app/non-static/gov_awards.sqlite')

# Create both the Stations and Measurements tables within the database
Base.metadata.create_all(engine)

# create session object
session = Session(bind=engine)

In [10]:
us_awards = us_data['results']

In [11]:
awards_list = []

for us_award in us_awards:
    
    award = Awards(
        awarding_agency = us_award['awarding_agency']['toptier_agency']['name'],
        date_signed = us_award['date_signed'],
        recipient_name = us_award['recipient']['recipient_name'],
        recipient_zip = us_award['recipient']['location']['zip5'],
        total_obligation = us_award['total_obligation']
    )
    
    awards_list.append(award)

In [12]:
# add stations instances to hawaii.sqlite database
session.bulk_save_objects(awards_list)

# commit changes
session.commit()

In [14]:
awards = (session.query(Awards).limit(5))

for award in awards:
    
    print(award.total_obligation)
    print(award.awarding_agency)

    print(award.recipient_name)

    print(award.recipient_zip)

    print(award.date_signed)



500.0
Department of Agriculture
FIRST SERVICE APPRAISAL INC
57201
2017-08-23
9048.0
Department of Agriculture
PIX4D INC
94105
2017-08-22
2500.0
Department of Agriculture
SECURITY TITLE AGENCY, INC.
85013
2017-06-07
144908.0
Department of Agriculture
MC CLURE CO INC
17110
2017-02-28
25589.0
Department of Agriculture
COMPUMERIC ENGINEERING, INC.
91761
2017-06-16
